In [42]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [46]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/Ret_Hirschsprung.pdf")
docs = loader.load()
docs

[Document(page_content='RESEARCH ARTICLE Open Access\nComprehensive analysis ofRET common and rare\nvariants in a series of Spanish Hirschsprung\npatients confirms a synergistic effect of both\nkinds of events\nRocio Núñez-Torres1,2, Raquel M Fernández1,2, Manuel Jesus Acosta1,2, Maria del Valle Enguix-Riego1,2,\nMartina Marbá3, Juan Carlos de Agustín4, Luis Castaño5, Guillermo Antiñolo1,2 and Salud Borrego1,2*\nAbstract\nBackground: RET is the major gene associated to Hirschsprung disease (HSCR) with differential contributions of its\nrare and common, coding and noncoding mutations to the multifactorial nature of this pathology. In the present\nstudy, we have performed a comprehensive study of our HSCR series evaluating the involvement of bothRET rare\nvariants (RVs) and common variants (CVs) in the context of the disease.\nMethods: RET mutational screening was performed by dHPLC and direct sequencing for the identification of RVs.\nIn addition Taqman technology was applied for the ge

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
textsplitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents= textsplitter.split_documents(docs)
documents[2].page_content

'the RET RV.\nConclusions: A gender effect exists on both the transmission and distribution of rare coding and common HSCR\ncausing mutations. In addition, theseRET CVs and RVs seem to act in a synergistic way leading to HSCR\nphenotype.\nBackground\nHirschsprung disease (HSCR, OMIM #142623) is a con-\ngenital malformation of the hindgut produced by a dis-\nruption in the neural crest cells (NCC) migration during\nembryonic development. This disorder results in an\nabsence of intramural ganglion cells in the submucosal\nand myenteric plexuses producing a functional intestinal\nobstruction. According to the aganglionic segment\nlength, patients could be classified as short-segment\nHSCR (S-HSCR), when aganglionosis extend as far as\nthe rectosigmoid junction, and long-segment HSCR (L-\nHSCR), when it extends beyond it. HSCR presents an\nestimated incidence of 1/5000 live births with sex-\ndependent penetrance and male predominance of 4:1\n[1,2]. It most commonly presents sporadically, a

In [48]:
documents

[Document(page_content='RESEARCH ARTICLE Open Access\nComprehensive analysis ofRET common and rare\nvariants in a series of Spanish Hirschsprung\npatients confirms a synergistic effect of both\nkinds of events\nRocio Núñez-Torres1,2, Raquel M Fernández1,2, Manuel Jesus Acosta1,2, Maria del Valle Enguix-Riego1,2,\nMartina Marbá3, Juan Carlos de Agustín4, Luis Castaño5, Guillermo Antiñolo1,2 and Salud Borrego1,2*\nAbstract\nBackground: RET is the major gene associated to Hirschsprung disease (HSCR) with differential contributions of its\nrare and common, coding and noncoding mutations to the multifactorial nature of this pathology. In the present\nstudy, we have performed a comprehensive study of our HSCR series evaluating the involvement of bothRET rare\nvariants (RVs) and common variants (CVs) in the context of the disease.\nMethods: RET mutational screening was performed by dHPLC and direct sequencing for the identification of RVs.', metadata={'source': 'docs/Ret_Hirschsprung.pdf', 'p

In [49]:
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template(""" 
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [50]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

db= Chroma.from_documents(documents,OpenAIEmbeddings())

In [51]:
## Vector database
query= "Obstructive sleep apnea (OSA) is a"
result=db.similarity_search(query)
result[0].page_content

'according to three titration procedures, as well as to compare the therapeutic efficacy of these titration methods.\nTrial registration: ClinicalTrials.gov, NCT03716648. Registered on 23 October 2018.\nKeywords: Mandibular advancement device, Sleep-disordered breathing, Apnea –hypopnea index\nBackground\nObstructive sleep apnea (OSA) is a prevalent public health\nissue, affecting up to 17% of adult women and 34% of\nadult men [ 1]. It is characterized by repetitive episodes of\npartial (hypopneas) or complete (apneas) upper airway ob-\nstruction during sleep, leading to nocturnal hypoxemia\nand sleep fragmentation [ 2]. The clinical daytime conse-\nquences associated with untreated OSA include excessive\ndaytime sleepiness (EDS), impaired cognitive performance\nand reduced quality of life. There is a strong correlation\nbetween OSA and both traffic and occupational accidents\n[3]. Furthermore, epidemiological studies provide object-\nive evidence that OSA is an independent risk factor

In [52]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

In [53]:
retriever = db.as_retriever()

In [54]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)

In [55]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [57]:
response=retrieval_chain.invoke({"input":"Is Hirschsprung disease a mendelian or a multifactorial disorder?" })
response["answer"]

'Based on the provided context, Hirschsprung disease is likely a multifactorial disorder. This is supported by the mention of both rare and common coding and noncoding mutations in the RET gene contributing to the disease phenotype. Additionally, the gender effect on the transmission and distribution of these mutations suggests a complex interplay of genetic factors in the development of Hirschsprung disease.'